In [1]:
import pandas as pd
import os 
from matplotlib import pyplot as plt
import numpy as np



#Decide what columns we need
A_csv = pd.read_csv('xaa.csv')
B_csv = pd.read_csv('xab.csv', header=None)
C_csv = pd.read_csv('xac.csv', header=None)
D_csv = pd.read_csv('xad.csv', header=None)
#Add correct index to split csv
B_csv.columns = A_csv.columns
C_csv.columns = A_csv.columns
D_csv.columns = A_csv.columns
#Merging
AB_csv = pd.merge(A_csv, B_csv,how='outer')

ABC_csv = pd.merge(AB_csv, C_csv,how='outer')

ABCD_csv = pd.merge(ABC_csv, D_csv,how='outer')

master_df = ABCD_csv

In [2]:
#Function to change GeoLoc value
def geo_switch(geo):
    try:
        [lat, long] = str(geo[0])[1:-1].split(',')
    except TypeError:
        [lat, long] = ['NaN', 'NaN']
    except ValueError:
        [lat, long] = str(geo)[1:-1].split(',')
    
    
    return [lat, long]

In [3]:
#change the master_df using apply
master_df['GeoLocation'] = master_df['GeoLocation'].apply(lambda x: geo_switch(x))


In [4]:
#Dropping the US lines Values
master_df = master_df.dropna(subset=['CityName', 'GeoLocation'])

#Dropping the "CrdPrv" lines
master_df = master_df[master_df.DataValueTypeID != "CrdPrv"]
master_df.shape

(14000, 13)

In [5]:
master_df = master_df.assign(Measure_pop = master_df['Data_Value'] / 100 * master_df['PopulationCount'])
master_df['Measure_pop'] = master_df['Measure_pop'].fillna(0).astype(int)
master_df.shape

(14000, 14)

In [6]:
#unique list of categories and states
cat_id = master_df['Category'].unique()
states = master_df['StateAbbr'].unique()
#dictionary to hold your data
df_dict = {}

#index through to grab dataframe for each category
for category in cat_id:
    category_group = master_df[master_df['Category'] == category]
#get category info for each state        
    for state in states:
        state_group =  category_group[category_group['StateAbbr'] == state]
#create keys for the df_dict and values for each state
        df_dict[category+' '+ state] = state_group


In [7]:
#Prevention dictionary for all states
prevent_state_df = pd.DataFrame()
prevent_dict = {}
for state in states:
#make keys the state abbr
    prevent_dict[state] = (df_dict.get('Prevention'+' '+state))
    prevent_state_df = prevent_state_df.append(prevent_dict[state])


In [8]:
#Unhealthy DateFrame for all states
unhealthy_state_df = pd.DataFrame()
#Unhealthy dictionary for all states
unhealthy_dict = {}
for state in states: 
#make keys the state abbr
    unhealthy_dict[state] = (df_dict.get('Unhealthy Behaviors'+' '+state))
    unhealthy_state_df = unhealthy_state_df.append(unhealthy_dict[state])
unhealthy_state_df.head(20)

,Year,StateAbbr,CityName,GeographicLevel,Category,Measure,DataValueTypeID,Data_Value_Type,Data_Value,PopulationCount,GeoLocation,CategoryID,MeasureId,Measure_pop
256,2016,AL,Birmingham,City,Unhealthy Behaviors,Binge drinking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,13.0,212237,"[33.5275663773, -86.7988174678]",UNHBEH,BINGE,27590
1460,2016,AL,Birmingham,City,Unhealthy Behaviors,Current smoking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,22.6,212237,"[33.5275663773, -86.7988174678]",UNHBEH,CSMOKING,47965
1958,2016,AL,Birmingham,City,Unhealthy Behaviors,No leisure-time physical activity among adults...,AgeAdjPrv,Age-adjusted prevalence,35.8,212237,"[33.5275663773, -86.7988174678]",UNHBEH,LPA,75980
2258,2016,AL,Birmingham,City,Unhealthy Behaviors,Obesity among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,42.6,212237,"[33.5275663773, -86.7988174678]",UNHBEH,OBESITY,90412
2554,2016,AL,Birmingham,City,Unhealthy Behaviors,Sleeping less than 7 hours among adults aged >...,AgeAdjPrv,Age-adjusted prevalence,41.8,212237,"[33.5275663773, -86.7988174678]",UNHBEH,SLEEP,88715
2913,2016,AL,Hoover,City,Unhealthy Behaviors,Binge drinking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,18.0,81619,"[33.3767602729, -86.8051937568]",UNHBEH,BINGE,14691
3282,2016,AL,Hoover,City,Unhealthy Behaviors,Current smoking among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,13.9,81619,"[33.3767602729, -86.8051937568]",UNHBEH,CSMOKING,11345
3438,2016,AL,Hoover,City,Unhealthy Behaviors,No leisure-time physical activity among adults...,AgeAdjPrv,Age-adjusted prevalence,20.2,81619,"[33.3767602729, -86.8051937568]",UNHBEH,LPA,16487
3531,2016,AL,Hoover,City,Unhealthy Behaviors,Obesity among adults aged >=18 Years,AgeAdjPrv,Age-adjusted prevalence,28.9,81619,"[33.3767602729, -86.8051937568]",UNHBEH,OBESITY,23587
3622,2016,AL,Hoover,City,Unhealthy Behaviors,Sleeping less than 7 hours among adults aged >...,AgeAdjPrv,Age-adjusted prevalence,32.8,81619,"[33.3767602729, -86.8051937568]",UNHBEH,SLEEP,26771


In [9]:
#Outcomes dictionary for all states
outcomes_state_df = pd.DataFrame()
outcomes_dict = {}

for state in states:
#make keys the state abbr
    outcomes_dict[state] = (df_dict.get('Health Outcomes'+' '+state))
    outcomes_state_df = outcomes_state_df.append(outcomes_dict[state])



In [10]:
def get_data(category_df):
    group_df = category_df.groupby(['StateAbbr', 'MeasureId']).sum()
    group_df = group_df.assign(true_st_percent = group_df['Measure_pop'] / group_df['PopulationCount']*100)
    group_df = group_df.drop(columns=['Data_Value', 'Year']).reset_index()
    return group_df

In [11]:
prevent_percent = get_data(prevent_state_df)
unhealthy_percent = get_data(unhealthy_state_df)
outcomes_percent = get_data(outcomes_state_df)
unhealthy_percent['MeasureId'].unique()

array(['BINGE', 'CSMOKING', 'LPA', 'OBESITY', 'SLEEP'], dtype=object)

In [12]:
binge = unhealthy_percent[unhealthy_percent['MeasureId'] == 'BINGE']
#binge.sort_values(by=['true_st_percent'], ascending=False).reset_index(drop=True)
binge['Rank'] = binge['true_st_percent'].rank(ascending=False)

c:\users\mason carnahan\documents\python_envs\this_works\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
#get ranks for each measureId 
def measure_rank(category_df):
    new_df = pd.DataFrame()
    measures = category_df['MeasureId'].unique()
    
    for measure in measures:
        measure_type = category_df[category_df['MeasureId'] == measure]
        measure_type['Rank'] = measure_type['true_st_percent'].rank(ascending=False)
        new_df = new_df.append(measure_type)
        




    return new_df

In [14]:
prevent_measure_rank = measure_rank(prevent_percent)
unhealthy_measure_rank = measure_rank(unhealthy_percent)
outcomes_measure_rank = measure_rank(outcomes_percent)


c:\users\mason carnahan\documents\python_envs\this_works\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [15]:
#get mean of ranks for each state
def avg_rank(measure_rank_df):
    rank_df = pd.DataFrame(index= states, columns=['Measure Rank'])
    for state in states:
        measure_avg = int((measure_rank_df[measure_rank_df['StateAbbr'] == state]['Rank']).mean())
        rank_df.loc[state, 'Measure Rank'] = measure_avg
    
    return rank_df.sort_values(by=['Measure Rank']).reset_index().rename(columns={'index': 'State'})

In [16]:
prevent_rank = avg_rank(prevent_measure_rank)
unhealthy_rank = avg_rank(unhealthy_measure_rank)
outcomes_rank = avg_rank(outcomes_measure_rank)

In [17]:
outcomes_rank.head()

,State,Measure Rank
0,MI,6
1,OH,8
2,TN,9
3,MS,9
4,AL,10


In [18]:
def true_rank(category_rank):
    out = pd.DataFrame(index=range(1, 52), columns= ['Rank'])
    for i in range(0, 51):
        out.iloc[i, 0] = (category_rank['State'][i])
    return out
def pretty_rank(true_rank):
    series = pd.Series(range(1, 52))
    rank_df = pd.DataFrame(index = true_rank.iloc[:, 0], columns=['Rank Value'])
    rank_df['Rank Value'] = series.values
    return rank_df

In [19]:
prevent_pretty_rank = pretty_rank(true_rank(prevent_rank))
unhealthy_pretty_rank = pretty_rank(true_rank(unhealthy_rank))
outcomes_pretty_rank = pretty_rank(true_rank(outcomes_rank))

In [20]:
prevent_pretty_rank

,Rank Value
Rank,
VA,1
WV,2
NC,3
AL,4
KY,5
SC,6
RI,7
DC,8
ME,9
